In [2]:
import os

In [3]:
%pwd

'/Users/sachinboora/Desktop/chicken-disease-classification-project/research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'/Users/sachinboora/Desktop/chicken-disease-classification-project'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:

from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
                self.config = read_yaml(config_filepath)
                self.params = read_yaml(params_filepath)

                create_directories([self.config.artifacts_root]) 


    def get_data_ingestion_config(self) -> DataIngestionConfig:
            config = self.config.data_ingestion

            create_directories([config.root_dir])

            data_ingestion_config = DataIngestionConfig(
                    root_dir=config.root_dir,
                    source_URL=config.source_URL,
                    local_data_file=config.local_data_file,
                    unzip_dir=config.unzip_dir,
            )   

            return data_ingestion_config

In [9]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename=self.config.local_data_file
                )
            logger.info(f"{filename} downloaded successfully with following info : {headers}")
        else:
            logger.info(f"file already exists of size : {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        '''
        zip_file_path : str
        Extracts the zip file into a data directory
        Function returns None
        '''

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e


[2024-04-02 10:16:50,002: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-04-02 10:16:50,003: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-02 10:16:50,004: INFO: common: created directory at artifacts]
[2024-04-02 10:16:50,004: INFO: common: created directory at artifacts/data_ingestion]
[2024-04-02 10:16:51,979: INFO: 2835700622: artifacts/data_ingestion/data.zip downloaded successfully with following info : Connection: close
Content-Length: 11747003
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "205720b5772ca1f86e2cc2af0a58ebc424d619351a3b79348c8ba8082408f1a4"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: F868:8A31D:27728E:3652EA:660B8DBA
Accept-Ranges: bytes
Date: Tue, 02 Apr 2024 04:46:51 GMT
Via: 1.1 varnish
X-Served-By: cache-del21720-DE